# LangGraph Advanced – Part 2️⃣: Data Cleaning Pipeline

ETL pipeline làm sạch CSV: dùng pandas + Claude để gợi ý sửa lỗi, sau đó áp dụng.

In [ ]:
%pip install -q langgraph langchain langchain-anthropic pandas tiktoken

In [ ]:
             import pandas as pd
             from langgraph.graph import StateGraph, MessageState
             from langchain_anthropic import ChatAnthropic
             from langchain.schema import HumanMessage

             CSV_PATH = "dirty_data.csv"
             OUTPUT_PATH = "clean_data.csv"
             llm = ChatAnthropic(model_name="claude-3-sonnet-20240229", temperature=0.2)

             class DataState(MessageState):
                 df: pd.DataFrame | None = None

             def load(_):
                 return DataState(df=pd.read_csv(CSV_PATH))

             def generate_instructions(state: DataState):
                 preview = state.df.head(20).to_csv(index=False)
                 prompt = f"""Phát hiện vấn đề chất lượng dữ liệu và viết code pandas để làm sạch:

CSV:
{preview}
"""
                 code_block = llm([HumanMessage(content=prompt)]).content
                 state.df.to_csv("_raw_preview.csv", index=False)
                 # Lưu hướng dẫn để dev xem, demo:
                 with open("clean_instructions.md", "w", encoding="utf-8") as f:
                     f.write(code_block)
                 return state

             g = StateGraph(DataState)
             g.add_node("load", load)
             g.add_node("analyze", generate_instructions)
             g.set_entry_point("load")
             g.add_edge("load", "analyze")
             g.compile().invoke(DataState())
